# Combine same variables

In [2]:
import pandas as pd
import os
import glob

# Your base directory
base_dir = '/Users/yuyiyang/Library/CloudStorage/Box-Box/data/AWARE data'

# Function to combine CSV files of the same type
def combine_csv_files_by_type(base_dir):
    # Dictionary to hold dataframes grouped by file type
    dfs = {}

    # Walk through the directory structure
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith('.csv'):
                # Extract the unique prefix and the variable type
                prefix = file.split('_')[0]
                variable_type = '_'.join(file.split('_')[1:])
                
                # Read the CSV file and add the prefix as a new column
                df = pd.read_csv(os.path.join(root, file))
                df.insert(0, 'Prefix', prefix)

                # If this variable type is not yet in our dictionary, add it
                if variable_type not in dfs:
                    dfs[variable_type] = df
                else:
                    # If it is, concatenate this dataframe with the existing one
                    dfs[variable_type] = pd.concat([dfs[variable_type], df], ignore_index=True)

    # Save each combined dataframe to a new CSV file
    for variable_type, df in dfs.items():
        output_path = os.path.join(base_dir, f'combined_{variable_type}')
        df.to_csv(output_path, index=False)
        print(f'Saved: {output_path}')

# Run the function
combine_csv_files_by_type(base_dir)


Saved: /Users/yuyiyang/Library/CloudStorage/Box-Box/data/AWARE data/combined_significant_motion.csv
Saved: /Users/yuyiyang/Library/CloudStorage/Box-Box/data/AWARE data/combined_locations.csv
Saved: /Users/yuyiyang/Library/CloudStorage/Box-Box/data/AWARE data/combined_plugin_device_usage.csv
Saved: /Users/yuyiyang/Library/CloudStorage/Box-Box/data/AWARE data/combined_sensor_accelerometer.csv
Saved: /Users/yuyiyang/Library/CloudStorage/Box-Box/data/AWARE data/combined_battery_discharges.csv
Saved: /Users/yuyiyang/Library/CloudStorage/Box-Box/data/AWARE data/combined_plugin_google_activity_recognition.csv
Saved: /Users/yuyiyang/Library/CloudStorage/Box-Box/data/AWARE data/combined_screen.csv
Saved: /Users/yuyiyang/Library/CloudStorage/Box-Box/data/AWARE data/combined_aware_log.csv
Saved: /Users/yuyiyang/Library/CloudStorage/Box-Box/data/AWARE data/combined_wifi.csv
Saved: /Users/yuyiyang/Library/CloudStorage/Box-Box/data/AWARE data/combined_calls.csv
Saved: /Users/yuyiyang/Library/CloudSt

# Location Prepocessing

## split to different coloumns

In [14]:
import json

# Load the dataset
df_loc = pd.read_csv('/Users/yuyiyang/Library/CloudStorage/Box-Box/data/AWARE data/combined_locations.csv')

# Define a function that converts the JSON string into a Python dictionary
def parse_json(data):
    try:
        return json.loads(data)
    except json.JSONDecodeError:
        return {}

# Apply the function to the 'CAST(data as CHAR)' column
expanded_data = df_loc['CAST(data as CHAR)'].apply(parse_json)

# Convert the resulting series of dictionaries to a DataFrame
expanded_df = pd.json_normalize(expanded_data)

# Concatenate the original DataFrame with the new columns from JSON
result_df_loc = pd.concat([df_loc.drop('CAST(data as CHAR)', axis=1), expanded_df], axis=1)
result_df_loc = result_df_loc.loc[:,~result_df_loc.columns.duplicated()]

# Save the resulting DataFrame to a new CSV file
result_df_loc.to_csv('/Users/yuyiyang/Library/CloudStorage/Box-Box/data/AWARE data/combined_locations_splited.csv', index=False)


## Get the top 5 locations for each participants

In [17]:
# Function to check if all intervals are the same for each group
def check_intervals(group):
    # Sort the timestamps
    sorted_timestamps = group.sort_values('timestamp')
    # Calculate the differences between consecutive timestamps
    intervals = sorted_timestamps['timestamp'].diff().dropna()
    # Check if all intervals are the same by checking the uniqueness of the interval values
    return intervals.nunique() == 1

# Apply the function to each group
intervals_consistency = result_df_loc.groupby('Prefix').apply(check_intervals)

# Print out the results
print(intervals_consistency)

Prefix
1020    False
1040    False
1167    False
1169    False
1200    False
        ...  
9673    False
9758    False
9771    False
9804    False
9985    False
Length: 156, dtype: bool


In [24]:
# Define a function to get the top 5 locations for each group
def top_five_locations(group):
    # Group by location and count occurrences
    location_counts = group.groupby(['double_latitude', 'double_longitude']).size().reset_index(name='counts')
    # Sort by counts in descending order
    top_locations = location_counts.sort_values('counts', ascending=False).head(5)
    return top_locations

# Apply the function to each participant's group of data
top_5_locations_each_person = result_df_loc.groupby('Prefix').apply(top_five_locations)

# Now `top_5_locations_each_person` is a multi-index DataFrame
# with the outer index being `Prefix` and the inner index being the original index from `groupby`.
# Clean up the index and leave `Prefix` as a column
top_5_locations_each_person = top_5_locations_each_person.reset_index(level='Prefix')

# Save the top_5_locations_each_person DataFrame to a CSV file
output_path = '/Users/yuyiyang/Library/CloudStorage/Box-Box/data/AWARE data/top_5_locations_each_person.csv'
top_5_locations_each_person.to_csv(output_path, index=False)

In [33]:
import requests
from urllib.parse import urlencode

def reverse_geocode(lat, lon):
    # Define the endpoint URL
    base_url = 'https://nominatim.openstreetmap.org/reverse?'
    # Define the parameters for the query
    params = {
        'format': 'jsonv2',
        'lat': lat,
        'lon': lon
    }
    # Encode the parameters and make the GET request
    response = requests.get(base_url + urlencode(params))
    # Parse the response to JSON
    data = response.json()
    # Extract the address and category
    address = data.get('display_name', 'Not Found')
    category = data.get('category', 'Not Found')
    return address, category

# Example usage:
lat, lon = 38.640744, -90.2302876
address, category = reverse_geocode(lat, lon)
print(address)
print(category)

John Cochran Division - VA St. Louis Health Care System, 915, North Grand Boulevard, Covenant Blu Grand Center, Saint Louis, Missouri, 63106, United States
amenity


In [34]:
import requests
from urllib.parse import urlencode

# Function to perform reverse geocoding
def reverse_geocode(lat, lon):
    base_url = 'https://nominatim.openstreetmap.org/reverse?'
    params = {
        'format': 'jsonv2',
        'lat': lat,
        'lon': lon
    }
    response = requests.get(base_url + urlencode(params))
    data = response.json()
    # Extract address and category; might need to adjust based on response structure
    address = data.get('display_name', 'Not Found')
    category = data.get('category', 'Not Found')
    return address, category

# Create new columns for address and category
top_5_locations_each_person['Address'] = ''
top_5_locations_each_person['Category'] = ''

for index, row in top_5_locations_each_person.iterrows():
    address, category = reverse_geocode(row['double_latitude'], row['double_longitude'])
    top_5_locations_each_person.at[index, 'Address'] = address
    top_5_locations_each_person.at[index, 'Category'] = category

# Since the process can be slow, it's recommended to print progress or use a loading bar library
print("Reverse geocoding completed.")

# Save the updated dataframe to a new CSV file
top_5_locations_each_person.to_csv('/Users/yuyiyang/Library/CloudStorage/Box-Box/data/AWARE data/updated_dataset.csv', index=False)


Reverse geocoding completed.


In [40]:
# Count the number of occurrences of each category
category_counts = top_5_locations_each_person['Category'].value_counts()

# Convert the Series to a DataFrame for easier handling 
category_counts_df = category_counts.reset_index()
category_counts_df.columns = ['Category', 'Counts']

category_counts_df

,Category,Counts
0,place,486
1,building,160
2,amenity,95
3,highway,24
4,leisure,14
5,tourism,1


- **place**: Geographic locations or features that don't fall neatly into more specific categories.
- **building**: Various types of structures that have a roof and walls, standing more or less permanently in one place. e.g. house, residential, etc.
- **amenity**: Includes facilities such as schools, banks, cinemas, parking, hospitals, etc.
- **highway**: Highway.
- **leisure**: Parks, sports centres, water parks, etc.
- **tourism**: Hotels, museums, attractions, etc.

## Reported Location Prepocessing

In [6]:
import pandas as pd
# Load the dataset
df_reportedloc = pd.read_csv('/Users/yuyiyang/Desktop/Psychology/AWARE/updated_activities_categories.csv')

/Users/yuyiyang/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [35]:
# Show all outputs of a cell in a jupyter notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [38]:
# Assuming df_reportedloc is your DataFrame
address_cols = [f'Address/location{i}' for i in range(1, 6)]
activity_cols = [f'Activity{i}_Category' for i in range(1, 6)]

# Melt the address columns
address_df = pd.melt(df_reportedloc, id_vars=['PID'], value_vars=address_cols, var_name='Location_Type', value_name='Address')

# Melt the activity category columns
activity_df = pd.melt(df_reportedloc, id_vars=['PID'], value_vars=activity_cols, var_name='Activity_Type', value_name='Activity_Category')

# Reset index to help with the merge
address_df = address_df.reset_index(drop=True)
activity_df = activity_df.reset_index(drop=True)

# Drop rows with NaN in 'Activity_Category'
activity_df = activity_df.dropna(subset=['Activity_Category'])

# Merge the two DataFrames based on the index
melted_df = pd.concat([activity_df,address_df['Address']], axis=1)

# Sort and view the result
melted_df = melted_df.sort_values(by=['PID', 'Activity_Type'])
melted_df = melted_df.dropna(subset=['Activity_Category'])
len(melted_df)
melted_df.head(15)

518

,PID,Activity_Type,Activity_Category,Address
99,1020.0,Activity1_Category,Personal Home,"1013 Tempo Dr, St. Louis, MO 63146"
282,1020.0,Activity2_Category,Work and Professional,"660 S Euclid Ave, Saint Louis, MO 63110"
465,1020.0,Activity3_Category,Travel and Transportation,"San Juan, Puerto Rico"
140,1040.0,Activity1_Category,Personal Home,"1909 S 8th St, Apt C, St. Louis, MO 63104"
323,1040.0,Activity2_Category,Work and Professional,"2520 January Ave, St. Louis, MO 63110"
104,1167.0,Activity1_Category,Personal Home,"5608 Pershing Ave. St. Louis, MO 63112"
287,1167.0,Activity2_Category,Work and Professional,"Whitaker Hall 1 Brookings Drive St. Louis, MO ..."
84,1169.0,Activity1_Category,Personal Home,"4525 lindell blv. St. Louis, MO, 63108"
267,1169.0,Activity2_Category,Education,"520 south Euclid Ave. St. Louis, MO, 63110"
450,1169.0,Activity3_Category,Personal Home,"5047 waterman blv. St. Louis, MO, 63108"


### API

In [ ]:
import requests
from urllib.parse import urlencode

def geocode(address):
    if address == 'N/A':
        return None, None
    base_url = 'https://nominatim.openstreetmap.org/search?'
    params = {'q': address, 'format': 'json'}
    try:
        response = requests.get(base_url + urlencode(params))
        data = response.json()
        if data:
            return data[0]['lat'], data[0]['lon']
    except requests.exceptions.JSONDecodeError:
        print(f"Error decoding JSON from response for address: {address}")
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
    return None, None

### convert from address to gps

In [ ]:
# Add empty columns for Latitude and Longitude
melted_df['Latitude'] = None
melted_df['Longitude'] = None

# Save progress every n iterations
n = 10

for index, row in melted_df.iterrows():
    address = row['Address']
    if address != 'N/A':
        latitude, longitude = geocode(address)
        melted_df.at[index, 'Latitude'] = latitude
        melted_df.at[index, 'Longitude'] = longitude

    # Save the DataFrame every n rows
    if index % n == 0:
        melted_df.to_csv('/Users/yuyiyang/Desktop/Psychology/AWARE/intermediate_result.csv', index=False)

# Save the final DataFrame after completing the loop (518 rows)
melted_df.to_csv('/Users/yuyiyang/Library/CloudStorage/Box-Box/SAIGE/data/AWARE Locations/reported_address_gps_droppedNA.csv', index=False)

In [ ]:
# Drop the location that cannot be convert
melted_df = melted_df.dropna(subset=['Latitude', 'Longitude'])
# Save the final DataFrame (317 rows)
melted_df.to_csv('/Users/yuyiyang/Desktop/Psychology/AWARE/reported_address_gps_droppedNA.csv', index=False)

# Match location from reported form to aware

In [76]:
# Load the dataset
df_loc = pd.read_csv('/Users/yuyiyang/Library/CloudStorage/Box-Box/SAIGE/data/AWARE data/combined_locations_splited.csv')
filtered_df = df_loc[['Prefix', 'timestamp', 'double_latitude', 'double_longitude']]
filtered_df = filtered_df.sort_values(by=['Prefix'])
filtered_df = filtered_df.rename(columns={
    'Prefix': 'PID',
    'double_latitude': 'Latitude',
    'double_longitude': 'Longitude'
})
filtered_df.head()
len(filtered_df)

,PID,timestamp,Latitude,Longitude
796467,1020,1.681854e+12,18.456244,-66.064731
795004,1020,1.681743e+12,18.455017,-66.063994
795003,1020,1.681742e+12,18.452613,-66.068781
795002,1020,1.681742e+12,18.450405,-66.070216
795001,1020,1.681742e+12,18.450405,-66.070216


1742032

In [77]:
# Load the dataset
melted_df = pd.read_csv('/Users/yuyiyang/Library/CloudStorage/Box-Box/SAIGE/data/AWARE Locations/reported_address_gps_droppedNA.csv')
melted_df.head()
len(melted_df)

,PID,Activity_Type,Activity_Category,Address,Latitude,Longitude
0,1020.0,Activity1_Category,Personal Home,"1013 Tempo Dr, St. Louis, MO 63146",38.676213,-90.457693
1,1020.0,Activity2_Category,Work and Professional,"660 S Euclid Ave, Saint Louis, MO 63110",38.634579,-90.263109
2,1020.0,Activity3_Category,Travel and Transportation,"San Juan, Puerto Rico",18.465299,-66.116666
3,1040.0,Activity2_Category,Work and Professional,"2520 January Ave, St. Louis, MO 63110",38.612187,-90.283699
4,1167.0,Activity1_Category,Personal Home,"5608 Pershing Ave. St. Louis, MO 63112",38.648370,-90.283262


317

In [79]:
# Check the data type of 'PID' column
pid_type = filtered_df['PID'].dtype
print(f"Data type of PID: {pid_type}")

pid_type2 = melted_df['PID'].dtype
print(f"Data type of PID: {pid_type2}")

Data type of PID: int64
Data type of PID: float64


In [80]:
# Convert PID column from float to int
melted_df['PID'] = melted_df['PID'].astype('int64')
melted_df.head()

,PID,Activity_Type,Activity_Category,Address,Latitude,Longitude
0,1020,Activity1_Category,Personal Home,"1013 Tempo Dr, St. Louis, MO 63146",38.676213,-90.457693
1,1020,Activity2_Category,Work and Professional,"660 S Euclid Ave, Saint Louis, MO 63110",38.634579,-90.263109
2,1020,Activity3_Category,Travel and Transportation,"San Juan, Puerto Rico",18.465299,-66.116666
3,1040,Activity2_Category,Work and Professional,"2520 January Ave, St. Louis, MO 63110",38.612187,-90.283699
4,1167,Activity1_Category,Personal Home,"5608 Pershing Ave. St. Louis, MO 63112",38.648370,-90.283262


In [81]:
def is_within_range(lat1, lon1, lat2, lon2, degree=0.0009):
    return abs(lat1 - lat2) <= degree and abs(lon1 - lon2) <= degree

# Iterate over each row in filtered_df
for index, filtered_row in filtered_df.iterrows():
    for _, melted_row in melted_df.iterrows():
        if filtered_row['PID'] == melted_row['PID'] and is_within_range(filtered_row['Latitude'], filtered_row['Longitude'], melted_row['Latitude'], melted_row['Longitude']):
            filtered_df.at[index, 'Activity_Category'] = melted_row['Activity_Category']
            break

len(filtered_df)
filtered_df.head()
filtered_df.to_csv('/Users/yuyiyang/Desktop/Psychology/AWARE/matched_self_reported_location.csv', index=False)

1742032

,PID,timestamp,Latitude,Longitude,Activity_Category
796467,1020,1.681854e+12,18.456244,-66.064731,NaN
795004,1020,1.681743e+12,18.455017,-66.063994,NaN
795003,1020,1.681742e+12,18.452613,-66.068781,NaN
795002,1020,1.681742e+12,18.450405,-66.070216,NaN
795001,1020,1.681742e+12,18.450405,-66.070216,NaN


In [82]:
unique_act = filtered_df['Activity_Category'].unique()
print(unique_act)

[nan 'Travel and Transportation' 'Personal Home' 'Work and Professional'
 'Health and Wellness' "Children's Activities" 'Education' 'Others Home'
 'Leisure and Recreation' 'Social and Community' 'Shopping and Services'
 'Restaurants and Cafes' 'Hospital']
